**МЕТОД КАНОНИЧЕСКИХ КОРРЕЛЯЦИЙ**

Вариант 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy

from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('LAB5_DATA_MSM.csv', decimal=',')
df.head()

,Y1,Y2,Y3,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
0,9.26,204.2,13.26,0.23,0.78,0.40,1.37,1.23,0.23,1.45,26006,167.69,47750,6.40,166.32,10.08,17.72
1,9.38,209.6,10.16,0.24,0.75,0.26,1.49,1.04,0.39,1.30,23935,186.10,50391,7.80,92.88,14.76,18.39
2,12.11,222.6,13.72,0.19,0.68,0.40,1.44,1.80,0.43,1.37,22589,220.45,43149,9.76,158.04,6.48,26.46
3,10.81,236.7,12.85,0.17,0.70,0.50,1.42,0.43,0.18,1.65,21220,169.30,41089,7.90,93.96,21.96,22.37
4,9.35,62.0,10.63,0.23,0.62,0.40,1.35,0.88,0.15,1.91,7394,39.53,14257,5.35,173.88,11.88,28.13


In [2]:
# Нормируем и центрируем данные
scaler = StandardScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=scaler.get_feature_names_out())

In [3]:
# В соответствии с вариантом: Y1, Y2, Y3, X1 – X5
Y = df.iloc[:, :3]
X = df.iloc[:, 3:8]

In [4]:
X.head()

,X1,X2,X3,X4,X5
0,-0.693444,0.844604,0.643130,0.220404,0.207189
1,-0.597734,0.275570,-0.267562,1.075143,-0.075520
2,-1.076282,-1.052176,0.643130,0.719002,1.055314
3,-1.267702,-0.672820,1.293625,0.576545,-0.983163
4,-0.693444,-2.190245,0.643130,0.077948,-0.313590


In [5]:
Y.head()

,Y1,Y2,Y3
0,0.498408,0.785691,-0.071066
1,0.544860,0.832312,-0.568133
2,1.601641,0.944547,0.002693
3,1.098412,1.066279,-0.136807
4,0.533247,-0.441986,-0.492771


## Задание 1

Найти оценки канонических переменных (найти канонические веса, использующиеся для вычисления значений канонических переменных) и канонических корреляций.

In [6]:
# Число наблюдений
n = X.shape[0]

# Число признаков в X
k = X.shape[1]

# Число признаков в Y
m = Y.shape[1]

# Ковариационные матрицы
A_11 = np.cov(Y, rowvar=False)
A_22 = np.cov(X, rowvar=False)
A_12 = (1/n) * np.array(Y).T @ np.array(X)

**Метод канонических корреляций**

Положим:

$$ u_i = \theta^{(i)T} \eta = \theta_1 \eta_1 + \theta_2 \eta_2 + \cdots + \theta_m \eta_m $$

$$ v_i = \beta^T \xi = \beta_1 \xi_1 + \beta_2 \xi_2 + \cdots + \beta_k \xi_k, \quad i = 1,..., m $$

Так как $\xi$ и $\eta$ центрированные величины, то:
$$ Eu_i = 0, \quad Ev_i = 0 $$  

Выберем $\theta^{(i)}$ и $\beta^{(i)}$ таким образом, чтобы:
$$ Du_i = 1, \quad Dv_i = 1 $$  
$$ D(u_i) = E(u_i^2) = \theta^{(i)T} A_{11} \theta^{(i)} = 1, \quad D(v_i) = E(v_i^2) = \beta^{(i)T} A_{22} \beta^{(i)} = 1 $$


Коэффициент корреляции между $u_i$ и $v_i$:
$$ \rho_{u_i, v_i} = E(u_i v_i) = \theta^{(i)T} A_{12} \beta^{(i)} $$

Векторы коэффициентов преобразования $\theta^{(i)}, i = 1,...,m $, удовлетворяющие условиям задачи поиска канонических переменных,
являются собственными векторами матрицы $A_{11}^{-1}A_{12}A_{22}^{-1}A_{12}^{T}$, соответствующими собственным значениям $λ_{1}^{2}, λ_{2}^{2}, ... , λ_{m}^{2}$.

Векторы $\beta^{(i)}, i = 1,...,m$ связаны с векторами $\theta^{(i)}$ соотношением:
$$\beta^{(i)} = \frac{1}{\lambda_{i}} A_{22}^{-1} A_{21} \theta^{(i)}, i=1,...,m$$

При этом $\rho_{i} = E(u_{i}v_{i}) = \lambda_{i}$.

---

**Нормирование векторов в соответствии с условием**

Если $v_j$ - ненормированный вектор, то:

$$ \alpha_j = c_j \cdot v_j $$

где $c_j$ - нормировочный коэффициент.

Подставляем:

$$ (c_j v_j)^{T} A (c_j v_j) = С $$

$$ c_j^2 \cdot v_j^{T} A v_j = С $$

$$ c_j = \sqrt{\frac{С}{v_j^{T} A v_j}} $$

In [7]:
# Найдем коэффициенты преобразования theta
eig_vals_squares, theta_vecs = np.linalg.eig(np.linalg.inv(A_11) @ A_12 @ np.linalg.inv(A_22) @ A_12.T)

# Сортировка по убыванию собственных значений
sorted_indices = np.argsort(eig_vals_squares)[::-1]
eig_vals_squares = eig_vals_squares[sorted_indices]
theta_vecs = theta_vecs[:, sorted_indices]

# Найдем коэффициенты beta
beta_vecs = eig_vals_squares**(-0.5) * (np.linalg.inv(A_22) @ A_12.T @ theta_vecs)

for i in range(m):

  # Нормируем theta в соответствии с условием
  C_theta = theta_vecs[:, i].T @ A_11 @ theta_vecs[:, i]
  theta_vecs[:, i] /= C_theta**0.5

  # Нормируем beta в соответствии с условием
  C_beta = beta_vecs[:, i].T @ A_22 @ beta_vecs[:, i]
  beta_vecs[:, i] /= C_beta**0.5

In [8]:
for j in range(m):
  theta_str = [f'({round(theta_vecs[i, j], 2)})*Y{i+1}' for i in range(theta_vecs.shape[0])]
  print(f'u{j+1} = ', end='')
  print(*theta_str, sep=' + ')

print()

for j in range(m):
  beta_str = [f'({round(beta_vecs[i, j], 2)})*X{i+1}' for i in range(beta_vecs.shape[0])]
  print(f'v{j+1} = ', end='')
  print(*beta_str, sep=' + ')

u1 = (-0.24)*Y1 + (0.59)*Y2 + (0.86)*Y3
u2 = (-0.46)*Y1 + (-0.59)*Y2 + (0.44)*Y3
u3 = (1.07)*Y1 + (-0.84)*Y2 + (0.24)*Y3

v1 = (0.21)*X1 + (0.27)*X2 + (0.29)*X3 + (-0.03)*X4 + (1.0)*X5
v2 = (1.09)*X1 + (0.31)*X2 + (0.12)*X3 + (-0.17)*X4 + (0.27)*X5
v3 = (-0.82)*X1 + (-0.49)*X2 + (-1.12)*X3 + (-0.66)*X4 + (0.21)*X5


In [9]:
# Переход в новые признаковые пространства
# Y_new = Y @ theta_vecs
# X_new = X @ beta_vecs

In [10]:
#new_corr_matrix = np.corrcoef(Y_new, X_new, rowvar=False)
#print('Корреляционная матрица для новых переменных:', end='\n\n')
#print(np.round(new_corr_matrix, 3))

In [11]:
# Вектор оценок канононических корреляций
corr_vec = np.diag(theta_vecs.T @ A_12 @ beta_vecs)
corr_vec

array([0.83639294, 0.6501883 , 0.17516018])

## Задание 2

Произвести оценку значимости полученных канонических корреляций и, соответственно, отсеять незначимые пары канонических переменных. Записать выражения для значимых канонических переменных через исходные признаки.

**Cтатистическая проверка значимости найденных канонических переменных**

$H_{0}: \rho_{p}=0, ... \rho_{m} = 0$

$$ \lambda = \left(\frac{\Delta_1}{\Delta_0}\right)^{n/2} = \left(\prod_{i=p}^m (1-\hat{\lambda}_i^2)\right)^{n/2} $$

$$ \eta = - \left\{ n - 1 - \frac{1}{2} \left( m + k + 1 \right) \right\} \cdot \ln \left( \prod_{i = p}^{m} \left( 1 - \lambda_i^2 \right) \right) → \chi^2$$

$$ \nu = (k - p + 1)(m - p + 1) $$

In [12]:
def corr_importance_test(n, m, k, eig_vals_squares, p, alpha=0.05):
  """
  args:
  n - число наблюдений
  k - число признаков в X
  m - число признаков в Y
  eig_vals_squares - собственные значения, упорядоченные по убыванию
  p - индекс, начиная с которого корреляции незначимы (индексация с 1)
  alpha - уровень значимости

  returns:
  chi_square - статистика
  chi_crit - квантиль
  p_value - достигнутый уровень значимости
  """

  # Статистика
  coef = n - 1 - 1/2 * (m + k + 1)
  ln = np.log(np.prod(1 - eig_vals_squares[p - 1:]))
  chi_square = -coef * ln

  # Степени свободы
  df = (k - p + 1) * (m - p + 1)

  # Квантиль
  chi_crit = chi2.ppf(1 - alpha, df)

  # Расчет достигнутого уровня значимости
  p_value = chi2.sf(chi_square, df)

  return chi_square, chi_crit, p_value

In [13]:
for p in range(1, m+1):
  chi_square, chi_crit, p_value = corr_importance_test(n, m, k, eig_vals_squares, p, alpha=0.05)
  if chi_square < chi_crit:
    print('Принимается нулевая гипотеза (уровень значимости: 0.05);')
    print(f'Все корреляции, начиная с {p}-й равны 0;')
    print(f'Достигнутый уровень значимости: {p_value}.')

Принимается нулевая гипотеза (уровень значимости: 0.05);
Все корреляции, начиная с 3-й равны 0;
Достигнутый уровень значимости: 0.6868531249282879.


Таким образом, значимыми являются только пары $\{u_1, v_1\}$ и $\{u_2, v_2\}$.

In [14]:
for j in range(m-1):
  theta_str = [f'({round(theta_vecs[i, j], 2)})*Y{i+1}' for i in range(theta_vecs.shape[0])]
  print(f'u{j+1} = ', end='')
  print(*theta_str, sep=' + ')

print()

for j in range(m-1):
  beta_str = [f'({round(beta_vecs[i, j], 2)})*X{i+1}' for i in range(beta_vecs.shape[0])]
  print(f'v{j+1} = ', end='')
  print(*beta_str, sep=' + ')

u1 = (-0.24)*Y1 + (0.59)*Y2 + (0.86)*Y3
u2 = (-0.46)*Y1 + (-0.59)*Y2 + (0.44)*Y3

v1 = (0.21)*X1 + (0.27)*X2 + (0.29)*X3 + (-0.03)*X4 + (1.0)*X5
v2 = (1.09)*X1 + (0.31)*X2 + (0.12)*X3 + (-0.17)*X4 + (0.27)*X5


## Задание 3

Найти корреляции между каноническими переменными и переменными из каждого
множества Y и X (координаты векторов канонических нагрузок).

In [15]:
# Столбцы - координаты векторов канонических нагрузок
cov_Y_U = A_11 @ theta_vecs[:, :m-1]
cov_Y_U

array([[ 0.19368664, -0.73898477],
       [ 0.50184548, -0.8408206 ],
       [ 0.86927675,  0.37007451]])

In [16]:
cov_X_V = A_22 @ beta_vecs[:, :m-1]
cov_X_V

array([[-0.2979385 ,  0.93891318],
       [ 0.44836736, -0.05445482],
       [-0.01811212, -0.46291366],
       [ 0.37653693, -0.35407979],
       [ 0.95931312, -0.05485545]])

## Задание 4

Вычислить извлеченную дисперсию каждой канонической переменной и совокупностью канонических переменных (для каждого множества) и определить избыточность каждого множества исходных данных.

Объясняемая (извлеченная) канонической переменной дисперсия:
$$ V_1^{(i)} = \| \hat{\Theta}^{(i)} \|^2, \quad V_2^{(i)} = \| \hat{B}^{(i)} \|^2, \quad i = \overline{1,m} $$

Доли суммарных дисперсий исходных признаков, объясняемые p каноническими
переменными:

* $\sum_{j=1}^{p} V_{1}^{(j)} / Tr(A_{11})$ - для первого множества;
* $\sum_{j=1}^{p} V_{2}^{(j)} / Tr(A_{22})$ - для второго множества.

Избыточность первого множества переменных при заданном втором множестве:

$$ \frac{1}{\mathrm{Tr}(A_{11})} \sum_{j=1}^p \rho_j^2 V^{(j)}_{1} $$

Избыточность второго множества переменных при заданном первом множестве:

$$ \frac{1}{\mathrm{Tr}(A_{22})} \sum_{j=1}^p \rho_j^2 V^{(j)}_{2} $$

In [17]:
U_var = np.sum(cov_Y_U**2, axis=0)
V_var = np.sum(cov_X_V**2, axis=0)

print('==================================================================================================================')
print(f'Дисперсия множества Y, объясняемая u1: {U_var[0]}')
print(f'Дисперсия множества Y, объясняемая u2: {U_var[1]}')
print(f'Дисперсия множества Y, извлеченная u1 и u2: {sum(U_var)}')
print('==================================================================================================================')
print(f'Дисперсия множества X, объясняемая v1: {V_var[0]}')
print(f'Дисперсия множества X, объясняемая v2: {V_var[1]}')
print(f'Дисперсия множества X, извлеченная v1 и v2: {sum(V_var)}')
print('==================================================================================================================')

redundancy_U_of_V = 1 / np.trace(A_11) * np.sum(eig_vals_squares[:m-1] * U_var[:m-1])
print(f'Избыточность первого множества (Y) переменных при заданном втором множестве (X): {redundancy_U_of_V}')

redundancy_V_of_U = 1 / np.trace(A_22) * np.sum(eig_vals_squares[:m-1] * V_var[:m-1])
print(f'Избыточность второго множества (X) переменных при заданном первом множестве (Y): {redundancy_V_of_U}')

print('==================================================================================================================')

Дисперсия множества Y, объясняемая u1: 1.0450054642752162
Дисперсия множества Y, объясняемая u2: 1.3900329055285399
Дисперсия множества Y, извлеченная u1 и u2: 2.435038369803756
Дисперсия множества X, объясняемая v1: 1.3521904088257817
Дисперсия множества X, объясняемая v2: 1.2271939672632193
Дисперсия множества X, извлеченная v1 и v2: 2.579384376089001
Избыточность первого множества (Y) переменных при заданном втором множестве (X): 0.43126186186445054
Избыточность второго множества (X) переменных при заданном первом множестве (Y): 0.287416549389008


В результате канонического корреляционного анализа были выявлены две значимые пары канонических переменных, указывающие на статистически значимую линейную связь между исходными множествами данных. Эти канонические переменные объясняют существенную часть дисперсии друг друга, демонстрируя взаимосвязь между двумя группами признаков.